# 1. Import thư viện các thư viện cần thiết:

In [21]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeRegressor
# Import the AdaBoostRegressor class from scikit-learn
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import OrdinalEncoder , StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error , mean_squared_error


# 2. Tải bộ dữ liệu: Các bạn tải bộ dữ liệu Housing.csv

In [2]:
%cd /content/drive/MyDrive/Colab Notebooks/MODULE03/data

/content/drive/MyDrive/Colab Notebooks/MODULE03/data


In [3]:
# Các bạn tải bộ dữ liệu Housing.csv
!gdown 1qeJqFtRdjjHqExbWJcgKy0yJbczTTAE3

Downloading...
From: https://drive.google.com/uc?id=1qeJqFtRdjjHqExbWJcgKy0yJbczTTAE3
To: /content/drive/MyDrive/Colab Notebooks/MODULE03/data/Housing.csv
100% 30.0k/30.0k [00:00<00:00, 99.8MB/s]


In [4]:
%cd ..

/content/drive/MyDrive/Colab Notebooks/MODULE03


# 3. Đọc bộ dữ liệu:

In [5]:
# Sử dụng thư viện pandas, chúng ta sẽ đọc file .csv lên như sau:
dataset_path = '/content/drive/MyDrive/Colab Notebooks/MODULE03/data/Housing.csv'
df = pd.read_csv ( dataset_path )

In [6]:
df

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,yes,no,no,no,yes,2,yes,furnished
1,12250000,8960,4,4,4,yes,no,no,no,yes,3,no,furnished
2,12250000,9960,3,2,2,yes,no,yes,no,no,2,yes,semi-furnished
3,12215000,7500,4,2,2,yes,no,yes,no,yes,3,yes,furnished
4,11410000,7420,4,1,2,yes,yes,yes,no,yes,2,no,furnished
...,...,...,...,...,...,...,...,...,...,...,...,...,...
540,1820000,3000,2,1,1,yes,no,yes,no,no,2,no,unfurnished
541,1767150,2400,3,1,1,no,no,no,no,no,0,no,semi-furnished
542,1750000,3620,2,1,1,yes,no,no,no,no,0,no,unfurnished
543,1750000,2910,3,1,1,no,no,no,no,no,0,no,furnished


# 4. Xử lý dữ liệu categorical:

In [7]:
categorical_cols = df.select_dtypes (include =[ 'object']).columns.to_list()
print ( categorical_cols )

['mainroad', 'guestroom', 'basement', 'hotwaterheating', 'airconditioning', 'prefarea', 'furnishingstatus']


In [8]:
# Sau khi đã xác định được đối tượng cần xử lý, ta sẽ sử dụng OrdinalEncoder() để chuyển đổi chúng thành dạng số như sau:
ordinal_encoder = OrdinalEncoder()
encoded_categorical_cols = ordinal_encoder.fit_transform( df[ categorical_cols ])
encoded_categorical_df = pd.DataFrame(encoded_categorical_cols ,columns = categorical_cols)
numerical_df = df.drop( categorical_cols , axis =1)
encoded_df = pd.concat([ numerical_df , encoded_categorical_df ], axis =1)


In [9]:
# Bộ dữ liệu Housing sau khi đã chuyển đổi toàn bộ các đặc trưng Categorical về dạng số
encoded_df

,price,area,bedrooms,bathrooms,stories,parking,mainroad,guestroom,basement,hotwaterheating,airconditioning,prefarea,furnishingstatus
0,13300000,7420,4,2,3,2,1.0,0.0,0.0,0.0,1.0,1.0,0.0
1,12250000,8960,4,4,4,3,1.0,0.0,0.0,0.0,1.0,0.0,0.0
2,12250000,9960,3,2,2,2,1.0,0.0,1.0,0.0,0.0,1.0,1.0
3,12215000,7500,4,2,2,3,1.0,0.0,1.0,0.0,1.0,1.0,0.0
4,11410000,7420,4,1,2,2,1.0,1.0,1.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
540,1820000,3000,2,1,1,2,1.0,0.0,1.0,0.0,0.0,0.0,2.0
541,1767150,2400,3,1,1,0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
542,1750000,3620,2,1,1,0,1.0,0.0,0.0,0.0,0.0,0.0,2.0
543,1750000,2910,3,1,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# 5. Chuẩn hóa bộ dữ liệu

In [10]:
# Để việc tính toán thuận lợi, ta tiến hành chuẩn hóa toàn bộ giá trị trong bộ dữ liệu sử dụng StandardScaler() như sau:
normalizer = StandardScaler()
dataset_arr = normalizer.fit_transform( encoded_df )


In [11]:
# Bộ dữ liệu Housing (ở dạng ndarray) sau khi được chuẩn hóa.
dataset_arr

array([[ 4.56636513,  1.04672629,  1.40341936, ...,  1.4726183 ,
         1.80494113, -1.40628573],
       [ 4.00448405,  1.75700953,  1.40341936, ...,  1.4726183 ,
        -0.55403469, -1.40628573],
       [ 4.00448405,  2.21823241,  0.04727831, ..., -0.67906259,
         1.80494113, -0.09166185],
       ...,
       [-1.61432675, -0.70592066, -1.30886273, ..., -0.67906259,
        -0.55403469,  1.22296203],
       [-1.61432675, -1.03338891,  0.04727831, ..., -0.67906259,
        -0.55403469, -1.40628573],
       [-1.61432675, -0.5998394 ,  0.04727831, ..., -0.67906259,
        -0.55403469,  1.22296203]])

# 6. Tách dữ liệu X, y

In [13]:
'''
Khi đã hoàn tất các bước tiền xử lý, lúc này ta sẽ tách dữ liệu ban đầu thành
hai biến X, y đại diện cho các đặc trưng và nhãn. Nhận thấy cột đầu tiên, price, là nhãn của bộ
dữ liệu, ta sẽ tiến hành tách X, y ra như sau:
X, y = dataset_arr [:, 1:] , dataset_arr [:, 0]
'''
X = dataset_arr[:,1:]
y = dataset_arr[:,0]

# 7. Chia tập dữ liệu train, val:

In [15]:
'''
Dựa vào bộ dữ liệu gốc, ta cần chia thành hai tập dữ liệu con,
một dùng cho việc huấn luyện mô hình và một cho việc đánh giá mô hình. Ở đây, ta sẽ chia theo
tỷ lệ 7:3 và tham số ngẫu nhiên random_state = 1:
'''
X_train , X_val , y_train , y_val = train_test_split(X,y,test_size=0.3,random_state=1, shuffle= True)

# 8. Huấn luyện mô hình:

In [16]:
'''
Ta thực hiện huấn luyện mô hình với bộ dữ liệu train.
'''
# Để huấn luyện mô hình Random Forest, các bạn sẽ sử dụng RandomForestRegressor():
RFregressor = RandomForestRegressor(random_state = 1 )
RFregressor.fit( X_train , y_train )

RandomForestRegressor(random_state=1)

In [19]:
# Để huấn luyện mô hình AdaBoost, các bạn sẽ sử dụng AdaBoostRegressor():
ABregressor = AdaBoostRegressor (random_state = 1)
ABregressor.fit( X_train , y_train )

AdaBoostRegressor(random_state=1)

In [22]:
# Để huấn luyện mô hình Gradient Boosting, các bạn sẽ sử dụng GradientBoostingRegressor():
GBregressor = GradientBoostingRegressor(random_state = 1)
GBregressor.fit( X_train , y_train )

GradientBoostingRegressor(random_state=1)

# 9. Đánh giá mô hình:

In [23]:
'''
Để biết được mô hình đã huấn luyện có hoạt động tốt trên các mẫu dữ liệu
mới hay không, ta sẽ đánh giá thông qua tập val. Đầu tiên, ta cho mô hình đã huấn luyện thực
hiện dự đoán trên toàn bộ tập val:
'''
y_pred_RF = RFregressor.predict( X_val )
y_pred_AB = ABregressor.predict( X_val )
y_pred_GB = GBregressor.predict( X_val )

In [25]:
'''
Lúc này, ta hoàn toàn có thể áp dụng những độ đo đánh giá dành cho bài Regression để thực hiện
đánh giá hiệu suất mô hình. Ở đây, ta sẽ sử dụng Mean Absolute Error (MAE) và Mean Squared Error (MSE):
'''
# Mean Absolute Error (MAE)
MAE_RF = mean_absolute_error( y_val , y_pred_RF )
MAE_AB = mean_absolute_error( y_val , y_pred_AB )
MAE_GB = mean_absolute_error( y_val , y_pred_GB )

MSE_RF = mean_squared_error( y_val , y_pred_RF )
MSE_AB = mean_squared_error( y_val , y_pred_AB )
MSE_GB = mean_squared_error( y_val , y_pred_GB )


In [30]:
print ('Evaluation results on validation set :')
print (f'Mean Absolute Error : { MAE_RF}')
print (f'Mean Squared Error : { MSE_RF}')

'''
print (f'Mean Absolute Error : { MAE_AB}')
print (f'Mean Squared Error : { MSE_AB}')
print (f'Mean Absolute Error : { MAE_GB}')
print (f'Mean Squared Error : { MSE_GB}')
'''

Evaluation results on validation set :
Mean Absolute Error : 0.46093873321571177
Mean Squared Error : 0.37944418523089524


"\nprint (f'Mean Absolute Error : { MAE_AB}')\nprint (f'Mean Squared Error : { MSE_AB}')\nprint (f'Mean Absolute Error : { MAE_GB}')\nprint (f'Mean Squared Error : { MSE_GB}')\n"